In [ ]:
#-------INSTALL BASEMAP ---------------------------------------
!sudo apt-get install python-grib 
#--------------------------------------------------------------
#       NOTAS
#!sudo python setup.py install
#!sudo apt-get install libgeos-dev 
#!sudo pip3 install -U git+https://github.com/matplotlib/basemap.git
#https://jswhit.github.io/pygrib/docs/--- leitura e gravação de arquivos GRIB 
#-------------------------------------------------------------------------
#Instalar o Pyproj
!pip install -q pyproj==2.3.0

#Install Basemap 
!apt-get install -q libgeos-dev

!pip install -q https://github.com/matplotlib/basemap/archive/master.zip

!pip install netCDF4

!pip install cartopy

!pip install metpy==1.0
#--------------------------------------------------CONSULTA---------------------
#https://unidata.github.io/python-gallery/examples/MSLP_temp_winds.html
#https://wrf-python.readthedocs.io/en/latest/plot.html
!pip install siphon


In [ ]:
#-----MAPAS-----#####################
!pip install cartopy
!pip install shapely --no-binary shapely --force
print('\n')

# Baixando o shapefile dos estados brasileiros
!wget -c https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2019/Brasil/BR/br_unidades_da_federacao.zip
print('\n')

# Descomprimindo o arquivo de shapefile dos estados brasileiros
!unzip -o br_unidades_da_federacao.zip
print('\n')

# Baixando o shapefile dos estados / províncias mundiais
!wget -c https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip
print('\n')

# Descomprimindo o arquivo de shapefile dos estados / províncias mundiais
!unzip -o ne_10m_admin_1_states_provinces.zip
print('\n')

# Download do script "utilities.py", do curso de processamento de satélites, com algumas funções que facilitam a manipulação de dados de satélite
!wget -c https://www.dropbox.com/s/i8j1g9xg1g7o56l/utilities.py
print('\n')

# Download da arquivo CPT exemplo (para realce de canais IR)
!wget -c https://www.dropbox.com/s/fdgnaqt91cy3x97/IR4AVHRR6.cpt
print('\n')

# Download da arquivo CPT exemplo (para realce de canais WV)
!wget -c https://www.dropbox.com/s/4l9fys6pv5c68xk/SVGAWVX_TEMP.cpt
print('\n')

###########################################################

In [ ]:
from datetime import datetime
from metpy.units import units
from netCDF4 import num2date
import numpy as np
import scipy.ndimage as ndimage
from siphon.ncss import NCSS
from matplotlib.cm import get_cmap
import matplotlib.colors as colors
import matplotlib as mpl
#-----METPY---------------------
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section
#from mpl_toolkits.axes_grid1 import make_axes_locatable
#----------------- DADO ERA5---------------------------------------------------- 
#Script para plotar os dados de msl+ vento  do ERA5: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from netCDF4 import Dataset
import pandas as pd
import xarray as xr

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader


import datetime as dt 
#import warnings
#warnings.filterwarnings("ignore")
from osgeo import gdal # Import the GDAL library
import matplotlib.cm as cm
from numpy import linspace
from numpy import meshgrid
import glob, os

In [ ]:
# DADO
#Abrir arquivo 
file =xr.open_dataset("/content/nivel_850_barracao.nc")
print(file)

<xarray.Dataset>
Dimensions:    (latitude: 281, longitude: 301, time: 12)
Coordinates:
  * longitude  (longitude) float32 -95.0 -94.75 -94.5 ... -20.5 -20.25 -20.0
  * latitude   (latitude) float32 10.0 9.75 9.5 9.25 ... -59.5 -59.75 -60.0
  * time       (time) datetime64[ns] 2016-07-13 ... 2016-07-15T18:00:00
Data variables:
    z          (time, latitude, longitude) float32 ...
    r          (time, latitude, longitude) float32 ...
    clwc       (time, latitude, longitude) float32 ...
    q          (time, latitude, longitude) float32 ...
    t          (time, latitude, longitude) float32 ...
    u          (time, latitude, longitude) float32 ...
    v          (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-10-28 14:02:41 GMT by grib_to_netcdf-2.23.0: /opt/ecmw...


In [ ]:
#Período a ser avaliado
date = list(pd.date_range('2016-07-13T00:00:00.00','2016-07-15T00:00:00.00', freq='6H').strftime('%Y-%m-%d %H'))

In [ ]:
# Extract the Brightness mslvalues from the NetCDF 
#(lon mais oeste; lat mais sul; lon mais leste e lat mais norte)     
extent = [-95.0,-60.0,-20.0,10.0]
min_lon = extent[0]; max_lon = extent[2]; min_lat = extent[1]; max_lat = extent[3]

# PLOT WITH ZOOMED DOMAIN
DPI = 300

In [ ]:
import matplotlib.colors as colors

In [ ]:
#-----------------------ADV DE TEMPERATURA---------------------#
for idx_tempo in range(len(date)):
  # variavel tempo: string
  ds = file.sel(time=date[idx_tempo]).metpy.parse_cf()
  #print(ds)
  #variaveis
  lat = ds.variables['latitude'][:].squeeze()
  lon = ds.variables['longitude'][:].squeeze()
  u = ds['u']
  v = ds['v']
#  w = ds['w'][:]
  q = ds['q'][:]
  z = ds['z'][:]
  t = ds['t']

# Convert winds to knots
  u_wind = ds['u'].values * units.knots
  v_wind = ds['v'].values * units.knots
#Convert winds to  m
  u_wind_m = units('m/s') * ds.variables['u'][:].squeeze()
  v_wind_m = units('m/s') * ds.variables['v'][:].squeeze()

# Combine 1D latitude and longitudes into a 2D grid of locations
  lon_2d, lat_2d = np.meshgrid(lon, lat)

   ##create a flag to determine wich barbs are flipped
  flip_flag = np.zeros((u_wind.shape[0],v_wind.shape[1]))  

#all flags below the equator will be flipped
  flip_flag[lat_2d <0] = 1 

# Use helper function defined above to calculate distance
# between lat/lon grid points
  dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)

# #CÁLCULO DA ADVECÇÃO 
  adv = mpcalc.advection(t-273.15, u, v)*(1e+04)
# Smooth heights and advection a little
# Be sure to only put in a 2D lat/lon or Y/X array for smoothing
  Z_850 = ndimage.gaussian_filter(z, sigma=3, order=0) * units.meter
  adv = ndimage.gaussian_filter(adv, sigma=3, order=0)

#-- create figure and axes object
  fig = plt.figure(figsize=(10,10))
  ax = plt.axes(projection=ccrs.PlateCarree())

  img_extent = [extent[0], extent[2], extent[1], extent[3]]
  ax.set_extent([extent[0], extent[2], extent[1], extent[3]], ccrs.PlateCarree())

  shapefile = list(shpreader.Reader ('ne_10m_admin_1_states_provinces.shp').geometries())
  ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray',facecolor='none', linewidth=0.3)

 # Add coastlines, borders and gridlines
  ax.coastlines(resolution='10m', color='black',linewidth=0.8)
  ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
  gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False

#------------------ADVECÇÃO-------------------------------------
# Evenly spacing latitudes and longitudes over a specified interval (n° of lines and columns)
  x = linspace(min_lon, max_lon, adv.shape[1])
  y = linspace(max_lat, min_lat, adv.shape[0]) 

# Create the rectangular grid out of these values
  xx, yy = meshgrid(x, y) 
# Define de contour interval
  min = np.min(adv)
  max = np.max(adv)

  interval = 0.5
  levels = np.arange(-3,3,interval)


#seleciona valores especificos para colorir diferente
  ncolors = list(plt.cm.bwr(np.linspace(0,1,len(levels))))
#intervalo desejado
  ncolors[5] = "white"
  ncolors[6] = "white"
  norm = colors.BoundaryNorm(levels,len(levels),ncolors)
  cmap = colors.ListedColormap(ncolors,"", len(ncolors))


  S2 = ax.contour(xx, yy, Z_850/10, alpha = 0.8, linewidth = 1.0, 
                  colors='black',width= 1.2,linestyles='dashed')
  plt.clabel(S2,inline=1,inline_spacing=8,fontsize=12,fmt='%1.0f',colors='black')
#shaded
  S1 = ax.contourf(xx, yy, adv, cmap=cmap ,levels=levels,norm=norm, extend='both')
  cb2=plt.colorbar(S1,ticks=levels,format='%.1f',pad=0.009,shrink=0.8)
  cb2.set_label('($10^{-4}$ °C $s^{-1}$)',fontsize=10)
#--------------------VENTO -----------------------------------------------------
# Barbelas 
  #plt.barbs(lon_2d[::10,::10], lat_2d[::10,::10], u_wind[::10,::10], v_wind[::10,::10],
   #flip_barb=flip_flag[::10,::10],length=6,fill_empty=False,sizes=dict(emptybarb=0., spacing=0.2, height=0.3))

#Vetor
  plt.quiver(lon_2d[::8,::8], lat_2d[::8,::8], u[::8,::8], v[::8,::8],color='gray')

#Linhas de corrente
  #plt.streamplot(lon_2d, lat_2d, u, v, density=2, linewidth=1.5, color='b', norm=None, arrowsize=1 )

# TITULOS 
  plt.title('Advecção de Temperatura ($10^{-4}$ °C $s^{-1}$) e Geopotencial (dam)', loc='center', fontsize=13,y=1.05)
  plt.title('ERA5 0.25°', loc='left', fontsize=9)
  plt.title('{}'.format(str(pd.to_datetime(str(ds.time.values), format='%Y-%m-%d %H'))  +  ' UTC'), fontsize=9,loc='right')

# EXPORT RESULT
  plt.savefig('{}'.format(str(pd.to_datetime(str(ds.time.values), format='%Y-%m-%d %H')) + "_adv_t.png"),dpi=DPI)


In [ ]:
#---------------ADVECÇÃO DE UMIDADE  -------------------------------------------
for idx_tempo in range(len(date)):
  # variavel tempo: string
  ds = file.sel(time=date[idx_tempo]).metpy.parse_cf()
  #print(ds)
  #variaveis
  lat = ds.variables['latitude'][:].squeeze()
  lon = ds.variables['longitude'][:].squeeze()
  u = ds['u']
  v = ds['v']
#  w = ds['w'][:]
  q = ds['q'][:]*1000
  z = ds['z'][:]
  t = ds['t']

# Convert winds to knots
  u_wind = ds['u'].values * units.knots
  v_wind = ds['v'].values * units.knots
#Convert winds to  m
  u_wind_m = units('m/s') * ds.variables['u'][:].squeeze()
  v_wind_m = units('m/s') * ds.variables['v'][:].squeeze()

# Combine 1D latitude and longitudes into a 2D grid of locations
  lon_2d, lat_2d = np.meshgrid(lon, lat)

   ##create a flag to determine wich barbs are flipped
  flip_flag = np.zeros((u_wind.shape[0],v_wind.shape[1]))  

#all flags below the equator will be flipped
  flip_flag[lat_2d <0] = 1 

# Use helper function defined above to calculate distance
# between lat/lon grid points
  dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)

# #CÁLCULO DA ADVECÇÃO 
  adv = mpcalc.advection(q, u, v)*(1e+04)
# Smooth heights and advection a little
# Be sure to only put in a 2D lat/lon or Y/X array for smoothing
  Z_850 = ndimage.gaussian_filter(z, sigma=3, order=0) * units.meter
  adv = ndimage.gaussian_filter(adv, sigma=3, order=0)

#-- create figure and axes object
  fig = plt.figure(figsize=(10,10))
  ax = plt.axes(projection=ccrs.PlateCarree())

  img_extent = [extent[0], extent[2], extent[1], extent[3]]
  ax.set_extent([extent[0], extent[2], extent[1], extent[3]], ccrs.PlateCarree())

  shapefile = list(shpreader.Reader ('ne_10m_admin_1_states_provinces.shp').geometries())
  ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray',facecolor='none', linewidth=0.3)

 # Add coastlines, borders and gridlines
  ax.coastlines(resolution='10m', color='black',linewidth=0.8)
  ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
  gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False

#-----------------------ADV----------------------------------------------------
# Evenly spacing latitudes and longitudes over a specified interval (n° of lines and columns)
  x = linspace(min_lon, max_lon, adv.shape[1])
  y = linspace(max_lat, min_lat, adv.shape[0]) 

# Create the rectangular grid out of these values
  xx, yy = meshgrid(x, y) 
# Define de contour interval
  min = np.min(adv)
  max = np.max(adv)

  interval = 0.5
  levels = np.arange(-2.5,2.5,interval)

#seleciona valores especificos para colorir diferente
  ncolors = list(plt.cm.RdYlGn(np.linspace(0,1,len(levels))))
#intervalo desejado
  ncolors[4] = "white"
  ncolors[5] = "white"
  norm = colors.BoundaryNorm(levels,len(levels),ncolors)
  cmap = colors.ListedColormap(ncolors,"", len(ncolors))

#Contour
  S2 = ax.contour(xx, yy, Z_850/10, alpha = 0.8,
                   linewidth = 1.0, colors='black',width= 1.2,linestyles='dashed')
  
  plt.clabel(S2,inline=1,inline_spacing=8,fontsize=12,fmt='%1.0f',colors='black')
  
#shaded
  S1 = ax.contourf(xx, yy, adv, cmap=cmap ,levels=levels,norm=norm, extend='both')
  cb2=plt.colorbar(S1,ticks=np.arange(-2.5,2.5,interval),format='%.1f',pad=0.009,shrink=0.8)
  cb2.set_label('($10^{-4}$ g $s^{-1}$ $kg^{-1}$)',fontsize=10)
#--------------------VENTO ---------------------------------------------------------------
# Barbelas 
  #plt.barbs(lon_2d[::10,::10], lat_2d[::10,::10], u_wind[::10,::10], v_wind[::10,::10],
   #flip_barb=flip_flag[::10,::10],length=6,fill_empty=False,sizes=dict(emptybarb=0., spacing=0.2, height=0.3))

#Plot como vetor
  plt.quiver(lon_2d[::8,::8], lat_2d[::8,::8], u[::8,::8], v[::8,::8],color='gray')

#Plot como linhas de corrente
  #plt.streamplot(lon_2d, lat_2d, u, v, density=2, linewidth=1.5, color='b', norm=None, arrowsize=1 )

# TITULOS 
  plt.title('Advecção de Umidade Específica ($10^{-4}$ g $s^{-1}$ $kg^{-1}$) e Geopotencial (dcam) ',
            loc='center', fontsize=12,y=1.05)
  plt.title('ERA5 0.25°', loc='left', fontsize=9)
  plt.title('{}'.format(str(pd.to_datetime(str(ds.time.values), format='%Y-%m-%d %H'))
    +  ' UTC'), fontsize=9,loc='right')

# EXPORT RESULT
  plt.savefig('{}'.format(str(pd.to_datetime(str(ds.time.values),
                                             format='%Y-%m-%d %H')) + "_adv_q.png"),dpi=DPI)

In [ ]:
#JBN
for idx_tempo in range(len(date)):
  # variavel tempo: string
  ds = file.sel(time=date[idx_tempo]).metpy.parse_cf()
  #print(ds)
  #variaveis
  lat = ds.variables['latitude'][:].squeeze()
  lon = ds.variables['longitude'][:].squeeze()
  u = ds['u']
  v = ds['v']
  #w = ds['w'][:]
  q = ds['q'][:]*1000
  z = ds['z'][:]
  t = ds['t']
  #CALCULO DO MODULO DO VENTO:
  mag_v= ((u**2)+(v**2))**(1/2)
# Convert winds to knots
  u_wind = ds['u'].values * units.knots
  v_wind = ds['v'].values * units.knots
#Convert winds to  m
  u_wind_m = units('m/s') * ds.variables['u'][:].squeeze()
  v_wind_m = units('m/s') * ds.variables['v'][:].squeeze()

# Combine 1D latitude and longitudes into a 2D grid of locations
  lon_2d, lat_2d = np.meshgrid(lon, lat)

   ##create a flag to determine wich barbs are flipped
  flip_flag = np.zeros((u_wind.shape[0],v_wind.shape[1]))  

#all flags below the equator will be flipped
  flip_flag[lat_2d <0] = 1 

# Use helper function defined above to calculate distance
# between lat/lon grid points
  dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)

# Smooth heights and advection a little
# Be sure to only put in a 2D lat/lon or Y/X array for smoothing
  Z_850 = ndimage.gaussian_filter(z, sigma=3, order=0) * units.meter

#-- create figure and axes object
  fig = plt.figure(figsize=(10,10))
  ax = plt.axes(projection=ccrs.PlateCarree())

  img_extent = [extent[0], extent[2], extent[1], extent[3]]
  ax.set_extent([extent[0], extent[2], extent[1], extent[3]], ccrs.PlateCarree())

  shapefile = list(shpreader.Reader ('ne_10m_admin_1_states_provinces.shp').geometries())
  ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray',facecolor='none', linewidth=0.3)

 # Add coastlines, borders and gridlines
  ax.coastlines(resolution='10m', color='black',linewidth=0.8)
  ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
  gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False


#-----------------------VENTO EM SHADED----------------------------------------------------
# Evenly spacing latitudes and longitudes over a specified interval (n° of lines and columns)
  x = linspace(min_lon, max_lon,  u.shape[1])
  y = linspace(max_lat, min_lat, u.shape[0]) 

# Create the rectangular grid out of these values
  xx, yy = meshgrid(x, y) 
# Define de contour interval
  min = np.min(mag_v)
  max = np.max(mag_v)
  interval = 2
#olhamos os valores do dado para que o 0 fique no branco
  levels = np.arange(12,max,interval)

#Contour
#S1 = plt.contourf(xx, yy, adv_q, cmap='RdYlGn' ,levels=levels)
  S1 = ax.contourf(xx, yy, mag_v, cmap='viridis_r' ,levels=levels,extend='max')
# Plot the contour labels
  plt.colorbar(S1,ticks=np.arange(12,max,interval),format='%.1f',pad=0.009,shrink=0.8)

  #Contour
  S2 = ax.contour(xx, yy, Z_850/10, alpha = 0.8, linewidth = 1.0, colors='darkred',
                   width= 1.2,linestyles='solid')
  
  cb2=plt.clabel(S2,inline=1,inline_spacing=8,fontsize=12,fmt='%1.0f',colors='darkred')
  #cb2.set_label('(m $s^{-1}$)',fontsize=10)
#--------------------VENTO ---------------------------------------------------------------
# Barbelas
  plt.barbs(lon_2d[::14,::14], lat_2d[::14,::14], u_wind[::14,::14], v_wind[::14,::14],
            flip_barb=flip_flag[::14,::14],
 length=5.5,fill_empty=False,sizes=dict(emptybarb=0., spacing=0.2, height=0.3),alpha=0.8)

#Vetor
#plt.quiver(lon_2d[::8,::8], lat_2d[::8,::8], u[::8,::8], v[::8,::8],color='gray')

#Linhas de corrente
#plt.streamplot(lon_2d, lat_2d, u, v, density=2, linewidth=1.5, color='b', norm=None, arrowsize=1 )

# TITULOS 
  plt.title('Vento (m $s^{-1}$) e Geopotencial (dcam)', loc='center', fontsize=12,y=1.05)
  plt.title('ERA5 0.25°', loc='left', fontsize=9)
  plt.title('{}'.format(str(pd.to_datetime(str(ds.time.values), format='%Y-%m-%d %H'))
    +  ' UTC'), fontsize=9,loc='right')

# EXPORT RESULT
  plt.savefig('{}'.format(str(pd.to_datetime(str(ds.time.values),
                                             format='%Y-%m-%d %H')) + "_vento.png"),dpi=DPI)
